# Loan Default Prediction

## Problem Statement

The objective of this project is to build a predictive model that can accurately determine whether a borrower will default on a loan or not. Using historical loan data, we aim to analyze the key factors influencing loan defaults and develop a machine learning model to predict the binary target variable: **Default (1: Defaulted, 0: Not Defaulted)**.

- **Loan_ID**: Unique Loan ID
- **Gender**: Male/ Female
- **Married**: Applicant married (Y/N)
- **Dependents**: Number of dependents
- **Education**: Applicant Education (Graduate/Under Graduate)
- **Self_Employed**: Self employed (Y/N)
- **ApplicantIncome**: Applicant income
- **CoapplicantIncome**: Coapplicant income
- **LoanAmount**: Loan amount in thousands
- **Loan_Amount_Term**: Term of loan in months
- **Credit_History**: Credit history meets guidelines
- **Property_Area**: Urban/ Semi Urban/ Rural
- **Loan_Status**: Loan approved (Y/N)

## Project Goals

1. **Data Preprocessing**:
    - Understand dataset
    - Handle missing values and outliers.
    - Convert categorical features into numerical representations.
    - Standardize numerical features for consistent model performance.
    - Explore correlations among features to understand relationships.

2. **Exploratory Data Analysis (EDA)**:
    - Visualize the distribution of key features such as `Income`, `LoanAmount`, and `CreditScore`.
    - Explore the relationship between features and loan default.
    - Examine the class distribution of the target variable (imbalanced or not).

3. **Feature Engineering**:
    - Engineer new features that may contribute to better prediction accuracy (e.g., `Debt-to-Income Ratio` buckets).
    - Perform feature selection based on correlation analysis, domain knowledge, and model performance.

4. **Modeling**:
    - Experiment with multiple machine learning models such as:
        - Logistic Regression
        - Decision Trees
        - Random Forest
        - Gradient Boosting Machines (GBM)
        - Support Vector Machines (SVM)
    - Tune hyperparameters to optimize performance using cross-validation.

5. **Evaluation Metrics**:
    - Measure model performance using metrics such as:
        - Accuracy
        - Precision
        - Recall
        - F1-score
        - ROC-AUC (Receiver Operating Characteristic - Area Under Curve)
    - Handle any class imbalance using techniques like SMOTE (Synthetic Minority Over-sampling Technique) or class weighting.

## Expected Outcomes

The project aims to deliver a predictive model with a high degree of accuracy in identifying potential loan defaulters. The model will help financial institutions assess borrower risk more effectively, reducing default rates and improving decision-making processes.

## Tools and Technologies

- **Python**: For data analysis and model development.
- **Pandas**: For data manipulation and analysis.
- **NumPy**: For numerical computations.
- **Matplotlib/Seaborn**: For data visualization.
- **Scikit-learn**: For model development, evaluation, and tuning.
- **Imbalanced-learn (if needed)**: For dealing with class imbalance.

## Conclusion

By leveraging machine learning models on borrower data, this notebook will predict the likelihood of a borrower defaulting on a loan. Insights gained from this analysis will provide actionable intelligence to stakeholders in the lending space, potentially reducing financial losses due to loan defaults.


----



#### Understanding the Dataset:

#### Imports

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Suppress all warnings
warnings.filterwarnings("ignore")

# Or to suppress specific warnings (e.g., FutureWarnings)
warnings.simplefilter(action="ignore", category=FutureWarning)

#### Load Datasets

In [ ]:
data = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
original_data = data.copy()
data.head(3)

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data.info()

### Missing Values Analysis

Analyzing missing values in a dataset is crucial for several reasons:

- **Data Quality**: Identifies columns with missing data, which may affect the quality and reliability of the analysis or model.
- **Data Cleaning**: Helps determine the extent of missing data, guiding decisions on imputation, removal, or other data cleaning methods.
- **Understanding Impact**: Provides insights into the data's completeness, allowing for better interpretation of results and ensuring that conclusions drawn are based on a robust dataset.
- **Model Performance**: Missing values can impact model performance; understanding their distribution helps in applying appropriate preprocessing techniques.

By calculating both the count and percentage of missing values, you gain a clearer picture of the dataset’s integrity and can make informed decisions on how to handle the missing data.


In [ ]:
# Calculate missing values and percentage
missing_values = data.isnull().sum()
missing_percentage = (missing_values / len(data)) * 100

# Round the percentages and format as strings
missing_summary = pd.DataFrame(
    {
        "Missing Values": missing_values,
        "Percentage": missing_percentage.round(2).astype(str) + "%",
    }
)

print("Missing Data Summary:")
display(missing_summary)

# Identify categorical and numerical features
categorical_features_train = data.select_dtypes(include=["object"]).columns.tolist()
categorical_features_test = test.select_dtypes(include=["object"]).columns.tolist()

numerical_features = data.select_dtypes(include=["float64", "int64"]).columns.tolist()

# Impute categorical features with mode
imputer_cat = SimpleImputer(strategy="most_frequent")
data[categorical_features_train] = imputer_cat.fit_transform(
    data[categorical_features_train]
)

# Impute numerical features with median
imputer_num = SimpleImputer(strategy="median")
data[numerical_features] = imputer_num.fit_transform(data[numerical_features])

# Verify missing values after imputation
missing_values_after = data.isnull().sum()
missing_percentage_after = (missing_values_after / len(data)) * 100

# Round the percentages and format as strings
missing_summary_after = pd.DataFrame(
    {
        "Missing Values": missing_values_after,
        "Percentage": missing_percentage_after.round(2).astype(str) + "%",
    }
)

print("\nMissing Data Summary After Imputation:")
display(missing_summary_after)

### Univariate Analysis
Perform univariate analysis by examining each variable individually. For categorical features, frequency tables or bar plots visualize category counts. For numerical variables, Probability Density Functions (PDFs) reveal their distributions.

##### Target Variable

The target variable, Default, is a categorical variable. A frequency table, percentage distribution, and bar plot will be used to analyze it. The frequency table provides the count of each category within the variable.

In [ ]:
# Set global plot style
sns.set(style="whitegrid")
plt.rcParams["axes.titlesize"] = 20  # Title font size
plt.rcParams["legend.fontsize"] = 15  # Legend font size
palette = [
    "#00bcd4",
    "#4169e1",
    "#003366",
    "#4682b4",
]  # Cyan, Royal Blue, Navy Blue, Steel Blue

sns.countplot(x="Loan_Status", data=data, palette=palette)
plt.show()

In [ ]:
# Univariate Analysis - Categorical Variables
fig, axes = plt.subplots(2, 3, figsize=(15, 12))

# Nominal variables
sns.countplot(data=data, x="Gender", ax=axes[0, 0], palette=palette)
axes[0, 0].set_title("Gender Distribution")

sns.countplot(data=data, x="Married", ax=axes[0, 1], palette=palette)
axes[0, 1].set_title("Marriage Status Distribution")

sns.countplot(data=data, x="Dependents", ax=axes[0, 2], palette=palette)
axes[0, 2].set_title("Dependents Number Distribution")

# Binary variables
sns.countplot(data=data, x="Education", ax=axes[1, 0], palette=palette)
axes[1, 0].set_title("Education Distribution")

sns.countplot(data=data, x="Self_Employed", ax=axes[1, 1], palette=palette)
axes[1, 1].set_title("Self_Employed status Distribution")

sns.countplot(data=data, x="Property_Area", ax=axes[1, 2], palette=palette)
axes[1, 2].set_title("Property Area Distribution")

plt.tight_layout()
plt.show()

In [ ]:
print(data["Education"].value_counts())

In [ ]:
# Calculate crosstabs
Education = pd.crosstab(data["Education"], data["Loan_Status"])
Married = pd.crosstab(data["Married"], data["Loan_Status"])
Dependents = pd.crosstab(data["Dependents"], data["Loan_Status"])
Self_Employed = pd.crosstab(data["Self_Employed"], data["Loan_Status"])
Gender = pd.crosstab(data["Gender"], data["Loan_Status"])
Property_Area = pd.crosstab(data["Property_Area"], data["Loan_Status"])

# Define the number of rows and columns for the plot grid
num_rows = 2
num_columns = 3

# Create subplots with the updated number of rows and columns
fig, axes = plt.subplots(
    num_rows, num_columns, figsize=(15, 12), constrained_layout=True
)

# Define the custom color palette
colors = ["#003366", "#3399FF"]  # Navy blue and normal blue

# Plot each cross-tabulation in specified subplot
Education.div(Education.sum(1).astype(float), axis=0).plot(
    kind="bar", stacked=True, ax=axes[0, 0], color=colors
)
axes[0, 0].set_title("Education Distribution")
axes[0, 0].set_xlabel("Education")
axes[0, 0].set_ylabel("Proportion")

Married.div(Married.sum(1).astype(float), axis=0).plot(
    kind="bar", stacked=True, ax=axes[0, 1], color=colors
)
axes[0, 1].set_title("Marital Status Distribution")
axes[0, 1].set_xlabel("Marital Status")
axes[0, 1].set_ylabel("Proportion")

Dependents.div(Dependents.sum(1).astype(float), axis=0).plot(
    kind="bar", stacked=True, ax=axes[0, 2], color=colors
)
axes[0, 2].set_title("Dependents Distribution")
axes[0, 2].set_xlabel("Dependents")
axes[0, 2].set_ylabel("Proportion")

Self_Employed.div(Self_Employed.sum(1).astype(float), axis=0).plot(
    kind="bar", stacked=True, ax=axes[1, 0], color=colors
)
axes[1, 0].set_title("Self_Employed Status Distribution")
axes[1, 0].set_xlabel("Self Employed Status")
axes[1, 0].set_ylabel("Proportion")

Gender.div(Gender.sum(1).astype(float), axis=0).plot(
    kind="bar", stacked=True, ax=axes[1, 1], color=colors
)
axes[1, 1].set_title("Gender Distribution")
axes[1, 1].set_xlabel("Gender")
axes[1, 1].set_ylabel("Proportion")

Property_Area.div(Property_Area.sum(1).astype(float), axis=0).plot(
    kind="bar", stacked=True, ax=axes[1, 2], color=colors
)
axes[1, 2].set_title("Property Area Distribution")
axes[1, 2].set_xlabel("Property Area")
axes[1, 2].set_ylabel("Proportion")

# Remove any unused subplots
# Since there are only 6 plots and we have a 2x3 grid, all subplots are used in this case.
# Ensure no unused subplot remains by deleting extra subplot axes if they exist
for ax in axes.flat:
    if not ax.has_data():  # Check if the axis has data; if not, remove it
        fig.delaxes(ax)

plt.show()

### Numerical Analysis

In this section, various plots are used to explore and understand the numerical features of the dataset.

- **Histograms** reveal the distribution and frequency of individual numerical variables.
- **Box Plots** provide insights into the spread, central tendency, and potential outliers within numerical features.
- **Pair Plots** examine the relationships between pairs of numerical features to identify correlations and interactions.
- **Correlation Heatmaps** visualize the strength and direction of linear relationships between numerical features.

### Analysis Type

The combination of these plots for numerical features extends beyond univariate analysis to include **bivariate** and **multivariate analysis**. Bivariate analysis examines the relationships between two variables, while multivariate analysis explores interactions among multiple variables simultaneously. This holistic approach provides a deeper understanding of the dataset's structure and relationships.

### 1. Histograms

In [ ]:
# Numerical features
numerical_features = data.select_dtypes(include=["float64", "int64"]).columns.tolist()

# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform the numerical features
numeric_scaled = scaler.fit_transform(data[numerical_features])

# Convert back to a DataFrame
numeric_scaled_df = pd.DataFrame(numeric_scaled, columns=numerical_features)

# Plot histograms for scaled features
fig, axes = plt.subplots(
    len(numerical_features), 1, figsize=(12, len(numerical_features) * 4)
)

for i, feature in enumerate(numerical_features):
    sns.histplot(
        numeric_scaled_df[feature], bins=20, kde=True, ax=axes[i], color="blue"
    )
    axes[i].set_title(f"Distribution of Scaled {feature}")
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel("Frequency")

plt.tight_layout()
plt.show()

In [ ]:
data["ApplicantIncome_log"] = np.log(data["ApplicantIncome"])
data["LoanAmount_log"] = np.log(data["LoanAmount"])
data["CoapplicantIncome_log"] = np.log(data["CoapplicantIncome"])

data["ApplicantIncome_log"].hist(bins=20)
plt.title("Histogram of Log-Transformed Applicant Income Distribution")
plt.show()
data["LoanAmount_log"].hist(bins=20)
plt.title("Histogram of Log-Transformed Coapplicant Income Distribution")
plt.show()
data["CoapplicantIncome"].hist(bins=20)
plt.show()

# Drop original features if log-transformed features are preferable
data.drop(["ApplicantIncome", "LoanAmount", "CoapplicantIncome"], axis=1, inplace=True)

data["ApplicantIncome_log"] = np.log(test["ApplicantIncome"])
data["LoanAmount_log"] = np.log(test["LoanAmount"])
data["CoapplicantIncome_log"] = np.log(test["CoapplicantIncome"])

test["ApplicantIncome_log"] = np.log(test["ApplicantIncome"])
test["LoanAmount_log"] = np.log(test["LoanAmount"])
test["CoapplicantIncome_log"] = np.log(test["CoapplicantIncome"])

# Drop original features if log-transformed features are preferable
test.drop(["ApplicantIncome", "LoanAmount", "CoapplicantIncome"], axis=1, inplace=True)

### 2. Box Plots

In [ ]:
numerical_features = data.select_dtypes(include=["float64", "int64"]).columns.tolist()

# Box plots for numerical variables
fig, axes = plt.subplots(
    len(numerical_features), 1, figsize=(12, len(numerical_features) * 3)
)
for i, feature in enumerate(numerical_features):
    sns.boxplot(x=data[feature], ax=axes[i], color="purple")
    axes[i].set_title(f"Box Plot of {feature}")
    axes[i].set_xlabel(feature)

plt.tight_layout()
plt.show()

### 3. Pair Plots


In [ ]:
# Pair plots for numerical variables
sns.pairplot(data[numerical_features], diag_kind="kde")
plt.suptitle("Pair Plot of Numerical Features", y=1.02)
plt.show()

### 4. Correlation Heatmap


Based on the heatmap below, here are some major deductions:

- A moderate positive correlation of 0.46 between Applicant Income and Loan Amount means that as people’s incomes go up, they generally borrow more money.

- A correlation of 0.19 between Coapplicant Income and Loan Amount shows a weak positive relationship, meaning that slightly higher coapplicant income is associated with a slightly larger loan amount, but the effect is minimal.

- A correlation of 0.12 between Loan Amount Term and Coapplicant Income indicates a very weak positive relationship, suggesting that coapplicant income has only a minimal impact on the length of the loan term.

In [ ]:
# Compute correlation matrix
corr_matrix = data[numerical_features].corr()

# Plot heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap="Blues", fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap of Numerical Features")
plt.show()

In [ ]:
data.sample(4)

In [ ]:
test.sample(3)

In [ ]:
data["CoapplicantIncome_log"].replace(-np.inf, 0, inplace=True)
data["ApplicantIncome_log"].replace(-np.inf, 0, inplace=True)
data["LoanAmount_log"].replace(-np.inf, 0, inplace=True)

test["CoapplicantIncome_log"].replace(-np.inf, 0, inplace=True)
test["ApplicantIncome_log"].replace(-np.inf, 0, inplace=True)
test["LoanAmount_log"].replace(-np.inf, 0, inplace=True)


# Impute categorical features with mode
imputer_cat = SimpleImputer(strategy="most_frequent")
data[categorical_features_train] = imputer_cat.fit_transform(
    data[categorical_features_train]
)
test[categorical_features_test] = imputer_cat.fit_transform(
    test[categorical_features_test]
)

# Impute numerical features with median
imputer_num = SimpleImputer(strategy="median")
data[numerical_features] = imputer_num.fit_transform(data[numerical_features])
test[numerical_features] = imputer_num.fit_transform(test[numerical_features])

##### Convert all categorical variables to numeric for modeling:

In [ ]:
# Encode the target variable 'Loan_Status'
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data["Loan_Status"])

# Drop 'Loan_ID' and 'Loan_Status' from the feature set
X = data.drop(["Loan_ID", "Loan_Status"], axis=1)
test_loan_id = test["Loan_ID"]
test = test.drop("Loan_ID", axis=1)

# Apply One-Hot Encoding to the categorical features
X = pd.get_dummies(X)
test = pd.get_dummies(test)

# Check the sample data
X.sample(5)

### Modeling

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "SVC": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting Classifier": GradientBoostingClassifier(),
}

# Set up cross-validation
cv = StratifiedKFold(n_splits=5)

# Evaluate models using cross-validation
results = {}
for model_name, model in models.items():
    cv_scores = cross_val_score(model, X, y, cv=cv, scoring="accuracy")
    results[model_name] = cv_scores
    print(
        f"{model_name}: Mean Accuracy = {cv_scores.mean():.4f}, Std = {cv_scores.std():.4f}"
    )

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix)

# Initialize Logistic Regression model
model = LogisticRegression(max_iter=1000)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
X_train.columns

In [ ]:
test.columns

In [ ]:
# Make predictions on the test set
predictions = model.predict(test)

# Create a DataFrame to hold predictions
submission = pd.DataFrame(
    {
        "Loan_ID": test_loan_id,  # Include this if it's part of the submission format
        "Loan_Status": predictions,
    }
)

# Save predictions to CSV for submission
submission.to_csv("data/my_submission.csv", index=False)

In [ ]:
submission.head(5)